In [1]:
import numpy as np
import open3d as o3d
import os
import glob
from data_understanding import read_meta_data,save_meta_data
# import pcl
import h5py

import enum
import os
import os.path as osp
import glob
import argparse
from itertools import product
from typing import List, Optional
import numpy as np
import torch

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def read_h5file(file_name):
    f = h5py.File(file_name, 'r')
    coords = np.array(f["xyz"])
    colors = np.array(f["colour"])
    semantic_labels = np.array(f["class_id"])
    instance_labels = np.array(f["ins_id"])
    f.close()
    indices = np.where(coords[:,2]>0)
    return coords[indices],colors[indices],semantic_labels[indices],instance_labels[indices]

In [3]:
# Copyright (c) Gorilla-Lab. All rights reserved.
import enum
import os
import os.path as osp
import glob
import argparse
from itertools import product
from typing import List, Optional
# import gorilla

import numpy as np
import torch


def sample_cloud(num_points, num_sample):
    if num_points >= num_sample:
        indices = np.random.choice(num_points, num_sample, replace=False)
    else:
        indices = np.random.choice(num_points,
                                   num_sample - num_points,
                                   replace=True)
        indices = list(range(num_points)) + list(indices)
    return indices


def room_to_blocks(coords: np.ndarray,
                   colors: np.ndarray,
                   semantic_labels: np.ndarray,
                   instance_labels: np.ndarray,
                   size: float = 1.0,
                   stride: float = 0.5,
                #    threshold: int = 4096,
                #    num_sample: Optional[int] = None,
                   verbose: bool = False) -> List:
    assert coords.shape[0] == colors.shape[0] == semantic_labels.shape[
        0] == instance_labels.shape[0]
    upper_bound = coords.max(axis=0)  # get upper bound of coordinates
    lower_bound = coords.min(axis=0)  # get lower bound of coordinates

    # partition into x-y axis blocks according to size and stride
    width = max(1,
                int(np.ceil((upper_bound[0] - lower_bound[0]) / stride + 1)))
    depth = max(1,
                int(np.ceil((upper_bound[1] - lower_bound[1]) / stride + 1)))
    cells = [xy for xy in product(range(width), range(depth))]

    if verbose:
        print(f"number of points: {coords.shape[0]}")

    # generate blocks
    block_list = []
    for (x, y) in cells:
        # get the inner block points' indices
        x_bound = lower_bound[0] if x==0 else (x * stride + lower_bound[0] - size)
        y_bound = lower_bound[1] if y==0 else (y * stride + lower_bound[1] - size)
        xcond = (coords[:, 0] >= x_bound) & (coords[:, 0] <= x_bound + size)
        ycond = (coords[:, 1] >= y_bound) & (coords[:, 1] <= y_bound + size)
        cond = xcond & ycond
        # filter out the meaningless block
        # if verbose:
        #     print(f"({x}, {y}): {cond.sum()}")
        # if np.sum(cond) < threshold:
        #     continue
        num_points = cond.sum()
        block_indices = np.where(cond)[0]
        # if num_sample is not None:
        #     sample_indices = sample_cloud(num_points, num_sample)
        #     block_indices = block_indices[sample_indices]
        block_coords = coords[block_indices]  # [num_block, 3]
        block_colors = colors[block_indices]  # [num_block, 3]
        block_semantic_labels = semantic_labels[block_indices,
                                                None]  # [num_block, 1]
        block_instance_labels = instance_labels[block_indices,
                                                None]  # [num_block, 1]
        # if block_instance_labels.max() < 0:
        #     continue
        concat_list = [
            block_coords, block_colors, block_semantic_labels,
            block_instance_labels
        ]

        block = np.concatenate(concat_list, axis=1)  # [num_block, 8/9]
        block_list.append(block)
    return block_list


# def get_parser():
#     parser = argparse.ArgumentParser(description="s3dis room partition "
#                                      "refer to jsis3d")
#     parser.add_argument("--data-dir",
#                         type=str,
#                         default="./inputs",
#                         help="directory save origin data")
#     parser.add_argument("--size",
#                         type=float,
#                         default=1,
#                         help="parition block size")
#     parser.add_argument("--stride",
#                         type=float,
#                         default=0.5,
#                         help="parition block stride")
#     parser.add_argument("--threshold",
#                         type=int,
#                         default=4096,
#                         help="parition number threshold")
#     parser.add_argument(
#         "--sample",
#         type=int,
#         default=None,
#         help="number of sample points, default is None(do not sample)")
#     parser.add_argument("--verbose",
#                         action="store_true",
#                         help="show partition information or not")

#     args_cfg = parser.parse_args()

#     return args_cfg


if __name__ == "__main__":
    
    # save_dir = f"{data_dir}_blocks"
    save_dir = f"data_blocks"

    os.makedirs(save_dir, exist_ok=True)
    for data_file in glob.glob("T**.h5")[5:]:
        scene = data_file.split(".")[0]
        # for data_file in glob.glob(osp.join(data_dir, "*.pth")):
        (coords, colors, semantic_labels, instance_labels) = read_h5file(data_file)
        # break
        # args.verbose
        if True:
            print(f"processing: {scene}")

        block_list = room_to_blocks(coords,colors,
                                    semantic_labels,
                                    instance_labels,
                                    size=100,
                                    stride=100*0.5,
                                    # threshold=args.threshold,
                                    # num_sample=args.sample,
                                    verbose=True
                                    )

        idx = 0
        for block in block_list:
            block_coords = block[:, 0:3]  # [num_block, 3]
            block_colors = block[:, 3:6]  # [num_block, 3]
            block_semantic_labels = block[:, 6]  # [num_block]
            if block_semantic_labels.max() < 2:
                continue
            if not ((block_coords.max(0) - block_coords.min(0)) > 0.5).all():
                continue
            block_instance_labels = block[:, 7]  # [num_block]
            scene_idx = f"{scene}_{idx}"
            idx += 1

            torch.save((block_coords, block_colors, block_semantic_labels,
                        block_instance_labels),
                       osp.join(save_dir, f"{scene_idx}.pth"))
        #     break
        # break


processing: T_316000_233500_NE_T_316000_233500_SW
number of points: 46589561


ValueError: zero-size array to reduction operation maximum which has no identity

In [7]:
    upper_bound = coords.max(axis=0)  # get upper bound of coordinates
    lower_bound = coords.min(axis=0)  # get lower bound of coordinates

    # partition into x-y axis blocks according to size and stride
    width = max(1,
                int(np.ceil((upper_bound[0] - lower_bound[0]) / 50 + 1)))
    depth = max(1,
                int(np.ceil((upper_bound[1] - lower_bound[1]) / 50 + 1)))
    cells = [xy for xy in product(range(width), range(depth))]

In [9]:
width

47

In [8]:
cells

[(0, 0),
 (0, 1),
 (0, 2),
 (0, 3),
 (0, 4),
 (0, 5),
 (0, 6),
 (0, 7),
 (0, 8),
 (0, 9),
 (0, 10),
 (0, 11),
 (0, 12),
 (0, 13),
 (0, 14),
 (0, 15),
 (0, 16),
 (0, 17),
 (0, 18),
 (0, 19),
 (0, 20),
 (0, 21),
 (0, 22),
 (0, 23),
 (0, 24),
 (0, 25),
 (0, 26),
 (0, 27),
 (0, 28),
 (0, 29),
 (0, 30),
 (0, 31),
 (0, 32),
 (0, 33),
 (0, 34),
 (0, 35),
 (0, 36),
 (0, 37),
 (0, 38),
 (0, 39),
 (0, 40),
 (0, 41),
 (0, 42),
 (0, 43),
 (0, 44),
 (0, 45),
 (0, 46),
 (0, 47),
 (0, 48),
 (0, 49),
 (0, 50),
 (0, 51),
 (0, 52),
 (0, 53),
 (0, 54),
 (0, 55),
 (0, 56),
 (0, 57),
 (0, 58),
 (0, 59),
 (0, 60),
 (0, 61),
 (0, 62),
 (0, 63),
 (0, 64),
 (0, 65),
 (0, 66),
 (0, 67),
 (0, 68),
 (0, 69),
 (0, 70),
 (0, 71),
 (0, 72),
 (0, 73),
 (0, 74),
 (0, 75),
 (0, 76),
 (1, 0),
 (1, 1),
 (1, 2),
 (1, 3),
 (1, 4),
 (1, 5),
 (1, 6),
 (1, 7),
 (1, 8),
 (1, 9),
 (1, 10),
 (1, 11),
 (1, 12),
 (1, 13),
 (1, 14),
 (1, 15),
 (1, 16),
 (1, 17),
 (1, 18),
 (1, 19),
 (1, 20),
 (1, 21),
 (1, 22),
 (1, 23),
 (1, 24),


In [17]:
"""processing: T_315500_233500_NE_T_315500_234000_SE
number of points: 12434593
processing: T_315500_234500_NE
number of points: 18220082
processing: T_315500_234500_NW
number of points: 42672312
processing: T_315500_234500_SE
number of points: 20845825
processing: T_315500_234500_SW
number of points: 16999037
processing: T_316000_233500_NE_T_316000_233500_SW
number of points: 46589561"""

57.0

In [106]:
all_check= np.append(block_coords,block_colors,axis=1)
np.savetxt("test_read50.txt",all_check,fmt="%1.8f %1.8f %1.8f %d %d %d")

In [76]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(block_coords)
pcd.colors = o3d.utility.Vector3dVector(block_colors)


In [77]:
o3d.visualization.draw_geometries([pcd])